In [2]:
# imports
import pandas as pd
import os # use this to access your environment variables
import requests # this will be used to call the APIs
import json

In [4]:
# From city bikes workbook
path = '/Users/parkerharalds/Documents/Lighthouse/Statistical_Modelling_Project/data/city_bikes.csv'
df = pd.read_csv(path)

In [ ]:
# smaller city bikes dataframe used for testing
df2 = df.head(2)
df2

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

**Function to make API calls based on latitude and longitude from city_bikes.csv:**

In [94]:
api_key = os.environ['FS_API_KEY']
headers = {"accept": "application/json",
           'Authorization': api_key}
url ='https://api.foursquare.com/v3/places/search?'

def get_nearby_places(lat, lon):
    filters = {  'll': f'{lat},{lon}',            #lat and lon to same string
                'radius': 1000,                  # Radius in meters (specified above)  
                'categories': 13032,       # code for coffee shops and cafes
                'limit': 50              
            }                 
    response = requests.get(url, headers= headers, params= filters)
    if response.status_code == 200:             #if api call was successfull return the results
        data = response.json()
        results = data['results']
        df_results = pd.json_normalize(results)
        return df_results
    else:
        print('Error')
        return None

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [95]:
# Iterate through the DataFrame and make API calls
df_all_stations = pd.DataFrame()
for index, row in df2.iterrows():
    lat, lon = row['latitude'], row['longitude']
    df_this_station = get_nearby_places(lat, lon)
    
    if df_this_station is not None:
        df_this_station['station'] = row['station_id']
        df_all_stations = pd.concat([df_all_stations, df_this_station], axis=0)

# df_all_stations

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Test api call:

In [155]:
yelp_api_key = os.environ['yelp_api']
y_headers = {"accept": "application/json",
           'Authorization': f'Bearer {yelp_api_key}'
}
filters = {     'latitude': 48.4, 
                'longitude': 123.4,           
                'radius': 1000     
            }      
y_url = f'https://api.yelp.com/v3/businesses/search?'
y_response = requests.get(y_url, headers=y_headers, params=filters)

In [156]:
testing = y_response.json()
testing['businesses'] #will use this for json_normalize in function

Yelp API call:

In [136]:
yelp_api_key = os.environ['yelp_api']
y_headers = {"accept": "application/json",
           'Authorization': f'Bearer {yelp_api_key}'
           }
y_url = f'https://api.yelp.com/v3/businesses/search?'

def get_yelp_places(lat, lon):
    filters = { 'latitude': lat, 
                'longitude': lon,           
                'radius': 1000,                    
                'categories': 'cafes',      
            }                 
    y_response = requests.get(y_url, headers=y_headers, params=filters)
    if y_response.status_code == 200:             
        data = y_response.json()
        results = data['businesses']
        df_results = pd.json_normalize(results)
        return df_results
    else:
        print('Error')
        return None


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [ ]:
# Iterate through the DataFrame and make API calls
df_yelp_stations = pd.DataFrame()
for index, row in df2.iterrows():
    lat = row['latitude']
    lon =  row['longitude']
    df_this_yelp = get_yelp_places(lat, lon)
    
    if df_this_yelp is not None:
        df_this_yelp['station'] = row['station_id']
        df_yelp_stations = pd.concat([df_yelp_stations, df_this_yelp], axis=0)
df_yelp_stations = df_yelp_stations.reset_index(drop=True)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating